#Import

In [ ]:
import pandas as pd
import glob

# ----- 1. ค้นหาไฟล์ทั้งหมดที่ตรงตามเงื่อนไข -----
# รูปแบบชื่อไฟล์คือ 'gpos_' ตามด้วยอะไรก็ได้ และลงท้ายด้วย '.xls'
# ใช้ glob ในการค้นหาไฟล์ในโฟลเดอร์ปัจจุบัน
file_pattern = 'gpos_20*.xlsx'
file_list = glob.glob(file_pattern)

# ----- 2. เรียงลำดับชื่อไฟล์ตามวันที่ -----
# เนื่องจากรูปแบบวันที่เป็น yyyy-mm-dd การเรียงลำดับชื่อไฟล์ตามตัวอักษร (lexicographical sort)
# จะให้ผลลัพธ์เหมือนการเรียงตามวันที่โดยอัตโนมัติ (เช่น 'gpos_2023-12-31.xls' จะมาก่อน 'gpos_2024-01-01.xls')
file_list.sort()

# แสดงรายชื่อไฟล์ที่พบและเรียงลำดับแล้ว
print("ไฟล์ที่จะถูกนำมารวม (เรียงตามลำดับ):")
for f in file_list:
    print(f"- {f}")

# ----- 3. อ่านและรวมไฟล์ -----
# สร้าง list ว่างเพื่อเก็บ DataFrame ของแต่ละไฟล์
all_dfs = []

# วนลูปเพื่ออ่านแต่ละไฟล์ใน list ที่เรียงลำดับไว้แล้ว
for file in file_list:
    try:
        df = pd.read_excel(file)
        all_dfs.append(df)
        print(f"อ่านไฟล์ '{file}' สำเร็จ")
    except Exception as e:
        print(f"ไม่สามารถอ่านไฟล์ '{file}' ได้: {e}")


# ----- 4. รวม DataFrame ทั้งหมดให้เป็นอันเดียว -----
# ตรวจสอบว่ามี DataFrame ให้อย่างน้อยหนึ่งอันก่อนทำการรวม
if all_dfs:
    # ใช้ pd.concat เพื่อรวม DataFrame ทั้งหมดใน list
    # ignore_index=True เพื่อสร้าง index ใหม่สำหรับ DataFrame ที่รวมแล้ว
    combined_df = pd.concat(all_dfs, ignore_index=True)

    print("\nรวมไฟล์ทั้งหมดสำเร็จ!")

    # ----- 5. แสดงผลลัพธ์เบื้องต้น -----
    print("\nข้อมูล 5 แถวแรกของ DataFrame ที่รวมแล้ว:")
    print(combined_df.head())

    print("\nข้อมูลสรุปของ DataFrame:")
    combined_df.info()

    # หากต้องการเซฟ DataFrame ที่รวมแล้วเป็นไฟล์ใหม่
    # combined_df.to_csv('combined_gpos_data.csv', index=False)
    # print("\nได้บันทึกไฟล์ combined_gpos_data.csv เรียบร้อยแล้ว")
else:
    print("\nไม่พบไฟล์ที่ตรงกับรูปแบบ 'gpos_*.xls' ในโฟลเดอร์นี้")

#EDA/Proprocess

In [ ]:
df

In [ ]:
returned_items_df = df[df['รายละเอียดรายการ'].str.contains('คืน', na=False)]
display(returned_items_df)

In [ ]:
df = df.drop(returned_items_df.index)
display(df)

In [ ]:
df = df[df['เลขที่ใบเสร็จ'] != '--']
display(df)

In [ ]:
display(df.isnull().sum())

In [ ]:
df.info()

In [ ]:
df['วันที่'] = pd.to_datetime(df['วันที่'])
df['เวลา'] = pd.to_datetime(df['เวลา'])
df['เลขที่ใบเสร็จ'] = pd.to_numeric(df['เลขที่ใบเสร็จ'], errors='coerce').fillna(0).astype('int64')
df['ยอดรวมทั้งหมด'] = pd.to_numeric(df['ยอดรวมทั้งหมด'], errors='coerce').fillna(0).astype('int64')

In [ ]:
df.info()

##แยกประเภท ก๋วยเตี๋ยว, ตามสั่ง, เครื่องเคียง, เครื่องดื่ม

In [ ]:
unique_items = df['รายละเอียดรายการ'].unique()
display(unique_items)

In [ ]:
def categorize_item(item):
    if 'ก๋วยเตี๋ยว' in item:
        return 'ก๋วยเตี๋ยว'
    elif item in ['ผัดคะน้า', 'กะเพราโบราณผัดพริกแห้ง', 'ข้าวผัด', 'กระเทียมพริกไทย', 'ผัดพริกแกง', 'ข้าวผัดน้ำพริกกุ้งแห้ง', 'ผัดพริกเกลือ', 'ผัดไข่เค็ม', 'ข้าวโปะไข่ดาว', 'ข้าวไข่เจียว', 'ข้าวไข่เจียวน้ำพริก', 'ข้าวไข่ข้น', 'ข้าวคะน้า', 'ข้าวกะเพราเต้าหู้']:
        return 'ตามสั่ง'
    elif item in ['เก๊กฮวยใส่แก้ว', 'น้ำเปล่า', 'เอสโคล่าเล็ก 330 มล', 'เอสโคล่าใหญ่ 465 มล', 'น้ำแข็ง', 'เป๊ปซี่ 340 มล', 'โออิชิ 400 มล', 'น้ำเปล่า พนง', 'เป็ปซี่ พนง', 'เอสใหญ่ พนง', 'เอสเล็ก พนง']:
        return 'เครื่องดื่ม'
    else:
        return 'เครื่องเคียง'

df['ประเภท'] = df['รายละเอียดรายการ'].apply(categorize_item)
display(df.head())

##เมนูไหนขายดีสุด

In [ ]:
item_counts = df.groupby('รายละเอียดรายการ')['จำนวน'].sum().sort_values(ascending=False)
display(item_counts.head(20))

In [ ]:
import requests
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import os

# --- 1. ดาวน์โหลดฟอนต์ด้วย Python ---
font_url = 'https://github.com/google/fonts/raw/main/ofl/sarabun/Sarabun-Regular.ttf'
font_filename = 'Sarabun-Regular.ttf'

# ตรวจสอบว่ามีไฟล์ฟอนต์อยู่แล้วหรือไม่ ถ้ายังไม่มีให้ดาวน์โหลด
if not os.path.exists(font_filename):
    print(f"กำลังดาวน์โหลดฟอนต์ {font_filename}...")
    try:
        response = requests.get(font_url)
        response.raise_for_status()  # เช็คว่าดาวน์โหลดสำเร็จหรือไม่
        with open(font_filename, 'wb') as f:
            f.write(response.content)
        print("ดาวน์โหลดฟอนต์สำเร็จ")
    except requests.exceptions.RequestException as e:
        print(f"เกิดข้อผิดพลาดในการดาวน์โหลด: {e}")
else:
    print(f"พบไฟล์ '{font_filename}' อยู่แล้ว ไม่ต้องดาวน์โหลดใหม่")

# --- 2. เพิ่มฟอนต์และตั้งค่า Matplotlib ---
if os.path.exists(font_filename):
    # เพิ่มฟอนต์ที่ดาวน์โหลดเข้ามาในระบบของ Matplotlib
    fm.fontManager.addfont(font_filename)

    # ตั้งค่าให้ Matplotlib ใช้ฟอนต์ Sarabun เป็นหลัก
    plt.rc('font', family='Sarabun')
    
    print("\n✅ ตั้งค่าฟอนต์สำเร็จ!")
    print("⚠️ โปรดรีสตาร์ท Kernel เพื่อให้การเปลี่ยนแปลงมีผล")
else:
    print(f"❌ ไม่พบไฟล์ฟอนต์ '{font_filename}' การตั้งค่าล้มเหลว")

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# Create a bar chart for the top 20 items
plt.figure(figsize=(12, 8))
item_counts.head(20).plot(kind='bar')
plt.title('Top 20 Bestselling Items by Quantity')
plt.xlabel('Item Name')
plt.ylabel('Quantity Sold')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

In [ ]:
item_price_sum = df.groupby('รายละเอียดรายการ')['ราคา'].sum().sort_values(ascending=False)
display(item_price_sum.head(20))

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# Create a bar chart for the top 20 items by revenue
plt.figure(figsize=(12, 8))
item_price_sum.head(20).plot(kind='bar')
plt.title('Top 20 Bestselling Items by Revenue')
plt.xlabel('Item Name')
plt.ylabel('Total Revenue')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

In [ ]:
noodle_items = df[df['ประเภท'] == 'ก๋วยเตี๋ยว']
noodle_item_counts = noodle_items.groupby('รายละเอียดรายการ')['จำนวน'].sum().sort_values(ascending=False)
display(noodle_item_counts)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# Create a bar chart for noodle item counts
plt.figure(figsize=(12, 8))
noodle_item_counts.plot(kind='bar')
plt.title('Noodle Item Counts')
plt.xlabel('Noodle Type')
plt.ylabel('Quantity Sold')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

In [ ]:
noodle_items = df[df['ประเภท'] == 'ก๋วยเตี๋ยว']
noodle_item_counts = noodle_items.groupby('รายละเอียดรายการ')['ราคา'].sum().sort_values(ascending=False)
display(noodle_item_counts)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# Create a bar chart for noodle item revenue
plt.figure(figsize=(12, 8))
noodle_item_counts.plot(kind='bar')
plt.title('Noodle Item Revenue')
plt.xlabel('Noodle Type')
plt.ylabel('Total Revenue')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

In [ ]:
noodle_items = df[df['ประเภท'] == 'ตามสั่ง']
noodle_item_counts = noodle_items.groupby('รายละเอียดรายการ')['จำนวน'].sum().sort_values(ascending=False)
display(noodle_item_counts)

noodle_items = df[df['ประเภท'] == 'ตามสั่ง']
noodle_item_counts = noodle_items.groupby('รายละเอียดรายการ')['ราคา'].sum().sort_values(ascending=False)
display(noodle_item_counts)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# Filter for 'ตามสั่ง' items
tamsang_items = df[df['ประเภท'] == 'ตามสั่ง']

# Group by item details and sum the quantity
tamsang_item_counts = tamsang_items.groupby('รายละเอียดรายการ')['จำนวน'].sum().sort_values(ascending=False)

# Create a bar chart for ตามสั่ง item counts
plt.figure(figsize=(12, 8))
tamsang_item_counts.plot(kind='bar')
plt.title('ตามสั่ง Item Counts')
plt.xlabel('ตามสั่ง Type')
plt.ylabel('Quantity Sold')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

# Group by item details and sum the revenue
tamsang_item_revenue = tamsang_items.groupby('รายละเอียดรายการ')['ราคา'].sum().sort_values(ascending=False)

# Create a bar chart for ตามสั่ง item revenue
plt.figure(figsize=(12, 8))
tamsang_item_revenue.plot(kind='bar')
plt.title('ตามสั่ง Item Revenue')
plt.xlabel('ตามสั่ง Type')
plt.ylabel('Total Revenue')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

In [ ]:
noodle_items = df[df['ประเภท'] == 'เครื่องดื่ม']
noodle_item_counts = noodle_items.groupby('รายละเอียดรายการ')['จำนวน'].sum().sort_values(ascending=False)
display(noodle_item_counts)

noodle_items = df[df['ประเภท'] == 'เครื่องดื่ม']
noodle_item_counts = noodle_items.groupby('รายละเอียดรายการ')['ราคา'].sum().sort_values(ascending=False)
display(noodle_item_counts)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# Filter for 'เครื่องดื่ม' items
drink_items = df[df['ประเภท'] == 'เครื่องดื่ม']

# Group by item details and sum the quantity
drink_item_counts = drink_items.groupby('รายละเอียดรายการ')['จำนวน'].sum().sort_values(ascending=False)

# Create a bar chart for เครื่องดื่ม item counts
plt.figure(figsize=(12, 8))
drink_item_counts.plot(kind='bar')
plt.title('เครื่องดื่ม Item Counts')
plt.xlabel('เครื่องดื่ม Type')
plt.ylabel('Quantity Sold')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

# Group by item details and sum the revenue
drink_item_revenue = drink_items.groupby('รายละเอียดรายการ')['ราคา'].sum().sort_values(ascending=False)

# Create a bar chart for เครื่องดื่ม item revenue
plt.figure(figsize=(12, 8))
drink_item_revenue.plot(kind='bar')
plt.title('เครื่องดื่ม Item Revenue')
plt.xlabel('เครื่องดื่ม Type')
plt.ylabel('Total Revenue')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

In [ ]:
noodle_items = df[df['ประเภท'] == 'เครื่องเคียง']
noodle_item_counts = noodle_items.groupby('รายละเอียดรายการ')['จำนวน'].sum().sort_values(ascending=False)
display(noodle_item_counts)

noodle_items = df[df['ประเภท'] == 'เครื่องเคียง']
noodle_item_counts = noodle_items.groupby('รายละเอียดรายการ')['ราคา'].sum().sort_values(ascending=False)
display(noodle_item_counts)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# Filter for 'เครื่องเคียง' items
side_items = df[df['ประเภท'] == 'เครื่องเคียง']

# Group by item details and sum the quantity
side_item_counts = side_items.groupby('รายละเอียดรายการ')['จำนวน'].sum().sort_values(ascending=False)

# Create a bar chart for เครื่องเคียง item counts
plt.figure(figsize=(12, 8))
side_item_counts.plot(kind='bar')
plt.title('เครื่องเคียง Item Counts')
plt.xlabel('เครื่องเคียง Type')
plt.ylabel('Quantity Sold')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

# Group by item details and sum the revenue
side_item_revenue = side_items.groupby('รายละเอียดรายการ')['ราคา'].sum().sort_values(ascending=False)

# Create a bar chart for เครื่องเคียง item revenue
plt.figure(figsize=(12, 8))
side_item_revenue.plot(kind='bar')
plt.title('เครื่องเคียง Item Revenue')
plt.xlabel('เครื่องเคียง Type')
plt.ylabel('Total Revenue')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

##เวลาไหนยอดขายเท่าไร

In [ ]:
import matplotlib.pyplot as plt

# Extract the hour from the 'เวลา' column
df['ชั่วโมง'] = df['เวลา'].dt.hour

# Group by hour and sum the total revenue
hourly_revenue = df.groupby('ชั่วโมง')['ราคา'].sum()

# Create a histogram
plt.figure(figsize=(10, 6))
plt.bar(hourly_revenue.index, hourly_revenue.values, width=0.8, align='center')
plt.xticks(hourly_revenue.index)
plt.xlabel('Time')
plt.ylabel('Sales')
plt.title('Sales sum in each time')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Extract the hour from the 'เวลา' column (already done in a previous cell)
# df['ชั่วโมง'] = df['เวลา'].dt.hour

# Group by hour and เลขที่ใบเสร็จ and sum the total revenue for each order
hourly_order_revenue = df.groupby(['ชั่วโมง', 'เลขที่ใบเสร็จ'])['ยอดรวมทั้งหมด'].sum().reset_index()

# Group by hour and calculate the average of the total revenue per order
hourly_average_order_revenue = hourly_order_revenue.groupby('ชั่วโมง')['ยอดรวมทั้งหมด'].mean()

# Create a histogram
plt.figure(figsize=(10, 6))
plt.bar(hourly_average_order_revenue.index, hourly_average_order_revenue.values, width=0.8, align='center')
plt.xticks(hourly_average_order_revenue.index)
plt.xlabel('Time')
plt.ylabel('Sales per order')
plt.title('AVG Sales per order')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

##โต๊ะไหนคนทานบ่อยสุด

In [ ]:
table_item_counts = df.groupby('หมายเลขโต๊ะ')['จำนวน'].sum().sort_values(ascending=False)
display(table_item_counts)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# Create a bar chart for table item counts
plt.figure(figsize=(12, 8))
table_item_counts.plot(kind='bar')
plt.title('Item Counts by Table Number')
plt.xlabel('Table Number')
plt.ylabel('Quantity Sold')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

In [ ]:
def categorize_table_type(table):
    if table.startswith('ต'):
        return 'ทานที่ร้าน'
    elif table.startswith('ถ'):
        return 'กลับบ้าน'
    else:
        return 'อื่นๆ'

df['ประเภทโต๊ะ'] = df['หมายเลขโต๊ะ'].apply(categorize_table_type)

table_type_counts = df.groupby('ประเภทโต๊ะ')['จำนวน'].sum().sort_values(ascending=False)
display(table_type_counts)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# Create a pie chart for table type counts
plt.figure(figsize=(8, 8))
plt.pie(table_type_counts, labels=table_type_counts.index, autopct='%1.1f%%', startangle=90)
plt.title('Item Counts by Table Type')
plt.axis('equal') # Equal aspect ratio ensures that pie is drawn as a circle.
plt.show()

##โต๊ะไหนทำรายได้สูงสุด

In [ ]:
table_revenue = df.groupby('หมายเลขโต๊ะ')['ราคา'].sum().sort_values(ascending=False)
display(table_revenue)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# Create a bar chart for table revenue
plt.figure(figsize=(12, 8))
table_revenue.plot(kind='bar')
plt.title('Revenue by Table Number')
plt.xlabel('Table Number')
plt.ylabel('Total Revenue')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

In [ ]:
# Group by table and order to get the total revenue per order
table_order_revenue = df.groupby(['หมายเลขโต๊ะ', 'เลขที่ใบเสร็จ'])['ยอดรวมทั้งหมด'].sum().reset_index()

# Group by table and calculate the average of the order totals
average_revenue_per_table_order = table_order_revenue.groupby('หมายเลขโต๊ะ')['ยอดรวมทั้งหมด'].mean().sort_values(ascending=False)

display(average_revenue_per_table_order)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# Create a bar chart for average revenue per table order
plt.figure(figsize=(12, 8))
average_revenue_per_table_order.plot(kind='bar')
plt.title('Average Revenue per Order by Table Number')
plt.xlabel('Table Number')
plt.ylabel('Average Revenue per Order')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

##ช่องทางจ่ายเงินไหนนิยมสุด แต่ละช่องทางเมนูไหนขายดีสุด

In [ ]:
# Create a new dataframe to store the consolidated revenue
consolidated_revenue = {'Cash': 0, 'QrPromptPay': 0}

# Iterate through each row in the original dataframe
for index, row in df.iterrows():
    payment_methods = row['ประเภทการชำระเงิน'].split(',')
    price = row['ราคา']
    num_methods = len(payment_methods)

    # Distribute the revenue among the payment methods
    for method in payment_methods:
        method = method.strip() # Remove leading/trailing whitespace
        if method in consolidated_revenue:
            consolidated_revenue[method] += price / num_methods
        # Optionally, handle other payment methods if needed
        # else:
        #     print(f"Unknown payment method: {method}")

# Convert the dictionary to a pandas Series for display
consolidated_revenue_series = pd.Series(consolidated_revenue).sort_values(ascending=False)

display(consolidated_revenue_series)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# Create a pie chart for consolidated revenue by payment method
plt.figure(figsize=(8, 8))
plt.pie(consolidated_revenue_series, labels=consolidated_revenue_series.index, autopct='%1.1f%%', startangle=90)
plt.title('Revenue by Payment Method')
plt.axis('equal') # Equal aspect ratio ensures that pie is drawn as a circle.
plt.show()

In [ ]:
# Filter the DataFrame for Cash payments
cash_df = df[df['ประเภทการชำระเงิน'] == 'Cash']

# Group by item details and sum the quantity for Cash payments
cash_item_counts = cash_df.groupby('รายละเอียดรายการ')['จำนวน'].sum().sort_values(ascending=False)

print("รายการสินค้าที่ขายดีสำหรับช่องทาง Cash:")
display(cash_item_counts.head(10))

# Filter the DataFrame for QrPromptPay payments
qrpromptpay_df = df[df['ประเภทการชำระเงิน'] == 'QrPromptPay']

# Group by item details and sum the quantity for QrPromptPay payments
qrpromptpay_item_counts = qrpromptpay_df.groupby('รายละเอียดรายการ')['จำนวน'].sum().sort_values(ascending=False)

print("\nรายการสินค้าที่ขายดีสำหรับช่องทาง QrPromptPay:")
display(qrpromptpay_item_counts.head(10))

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# Create a bar chart for the top 10 best-selling items for Cash payments
plt.figure(figsize=(12, 8))
cash_item_counts.head(10).plot(kind='bar')
plt.title('Top 10 Bestselling Items (Cash)')
plt.xlabel('Item Name')
plt.ylabel('Quantity Sold')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

# Create a bar chart for the top 10 best-selling items for QrPromptPay payments
plt.figure(figsize=(12, 8))
qrpromptpay_item_counts.head(10).plot(kind='bar')
plt.title('Top 10 Bestselling Items (QrPromptPay)')
plt.xlabel('Item Name')
plt.ylabel('Quantity Sold')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

##ลูกค้าที่มาคนเดียว กับลูกค้าที่มาเป็นกลุ่ม มีพฤติกรรมการสั่งอาหารแตกต่างกันอย่างไร?

In [ ]:
# Separate data for single customers and groups
single_customers_df = df[df['จำนวนลูกค้า'] == 1]
group_customers_df = df[df['จำนวนลูกค้า'] > 1]

# Analyze popular items for single customers
single_customer_item_counts = single_customers_df.groupby('รายละเอียดรายการ')['จำนวน'].sum().sort_values(ascending=False)

print("รายการสินค้าที่ขายดีสำหรับลูกค้าที่มาคนเดียว:")
display(single_customer_item_counts.head(20))

# Analyze popular items for group customers
group_customer_item_counts = group_customers_df.groupby('รายละเอียดรายการ')['จำนวน'].sum().sort_values(ascending=False)

print("\nรายการสินค้าที่ขายดีสำหรับลูกค้าที่มาเป็นกลุ่ม:")
display(group_customer_item_counts.head(20))

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# Create a bar chart for the top 20 best-selling items for single customers
plt.figure(figsize=(12, 8))
single_customer_item_counts.head(20).plot(kind='bar')
plt.title('Top 20 Bestselling Items (Single Customers)')
plt.xlabel('Item Name')
plt.ylabel('Quantity Sold')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

# Create a bar chart for the top 20 best-selling items for group customers
plt.figure(figsize=(12, 8))
group_customer_item_counts.head(20).plot(kind='bar')
plt.title('Top 20 Bestselling Items (Group Customers)')
plt.xlabel('Item Name')
plt.ylabel('Quantity Sold')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

##วันแบบไหนที่โต๊ะ 11 จะถูกใช้เยอะสุด (ต้องเชื่อมกับข้อมูลสภาพแวดล้อม) พร้อมสำหรับการเพิ่มราคาหรือยัง

In [ ]:
# Filter data for table 'ต11'
table_11_df = df[df['หมายเลขโต๊ะ'] == 'ต11']

# Get the number of unique dates for table 'ต11'
unique_dates_table_11 = table_11_df['วันที่'].nunique()

# Get the total number of unique dates in the dataset
all_unique_dates = df['วันที่'].nunique()

# Calculate the percentage
percentage_table_11_used = (unique_dates_table_11 / all_unique_dates) * 100

print(f"เปอร์เซ็นต์ของวันที่ที่โต๊ะ ต11 ถูกใช้งาน: {percentage_table_11_used:.2f}%")

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# Calculate the number of days table 'ต11' was not used
days_table_11_not_used = all_unique_dates - unique_dates_table_11

# Create a Series for the pie chart
pie_data_usage = pd.Series({'โต๊ะ ต11 ถูกใช้งาน': unique_dates_table_11, 'โต๊ะ ต11 ไม่ถูกใช้งาน': days_table_11_not_used})

# Create a pie chart
plt.figure(figsize=(8, 8))
plt.pie(pie_data_usage, labels=pie_data_usage.index, autopct='%1.1f%%', startangle=90)
plt.title('Table ต11 Usage Proportion by Day')
plt.axis('equal') # Equal aspect ratio ensures that pie is drawn as a circle.
plt.show()

##ยอดขายเฉลี่ยต่อลูกค้าหนึ่งคน (Average Spend per Customer) เป็นเท่าไหร่ และจะเพิ่มได้อย่างไร?

In [ ]:
# Filter the DataFrame to include only 'ก๋วยเตี๋ยว' or 'ตามสั่ง' items
filtered_df = df[df['ประเภท'].isin(['ก๋วยเตี๋ยว', 'ตามสั่ง'])]

# Calculate total revenue for the filtered items
total_revenue_filtered = filtered_df['ราคา'].sum()

# Calculate total number of customers for the filtered orders
# We need to consider unique orders within the filtered data
total_customers_filtered = filtered_df.groupby('เลขที่ใบเสร็จ')['จำนวนลูกค้า'].first().sum()


# Calculate average spend per customer for the filtered items
average_spend_per_customer_filtered = total_revenue_filtered / total_customers_filtered

print(f"ยอดขายเฉลี่ยต่อลูกค้าหนึ่งคน (เฉพาะรายการก๋วยเตี๋ยวและตามสั่ง): {average_spend_per_customer_filtered:.2f}")

In [ ]:
import matplotlib.pyplot as plt

# Calculate total revenue per order
order_revenue = df.groupby('เลขที่ใบเสร็จ')['ราคา'].sum()

# Get the number of customers per order
order_customers = df.groupby('เลขที่ใบเสร็จ')['จำนวนลูกค้า'].first()

# Calculate revenue per customer for each order
revenue_per_customer_per_order = order_revenue / order_customers

# Add the hour to the revenue per customer series
revenue_per_customer_per_order = pd.merge(revenue_per_customer_per_order.rename('revenue_per_customer'),
                                          df.groupby('เลขที่ใบเสร็จ')['ชั่วโมง'].first(),
                                          left_index=True, right_index=True)

# Calculate the average revenue per customer per hour
average_revenue_per_customer_per_hour = revenue_per_customer_per_order.groupby('ชั่วโมง')['revenue_per_customer'].mean()

# Create a bar chart for hourly average revenue per customer
plt.figure(figsize=(10, 6))
bars = plt.bar(average_revenue_per_customer_per_hour.index, average_revenue_per_customer_per_hour.values, width=0.8, align='center')
plt.xticks(average_revenue_per_customer_per_hour.index)
plt.xlabel('Time')
plt.ylabel('Average Sales per Customer')
plt.title('Average Sales per Customer by Hour')
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Add the overall average spend per customer as text on the plot
# Ensure average_spend_per_customer_filtered is defined (from a previous cell)
if 'average_spend_per_customer_filtered' in locals():
    plt.text(0.5, 0.95, f"ยอดขายเฉลี่ยต่อลูกค้าหนึ่งคน (เฉพาะรายการก๋วยเตี๋ยวและตามสั่ง): {average_spend_per_customer_filtered:.2f}",
             horizontalalignment='center', verticalalignment='center', transform=plt.gca().transAxes,
             bbox=dict(boxstyle='round,pad=0.5', fc='wheat', alpha=0.5))

plt.show()

#เมนูไหนที่คนมักจะเพิ่มหรือสั่งพิเศษ

In [ ]:
# Filter the DataFrame to include only 'ตามสั่ง' items
tamsang_df = df[df['ประเภท'] == 'ตามสั่ง']

# Calculate the average price of 'ตามสั่ง' items
average_tamsang_price = tamsang_df['ราคา'].mean()

print(f"ราคาเฉลี่ยของรายการที่เป็น ตามสั่ง: {average_tamsang_price:.2f}")

In [ ]:
items_to_clean = ['ก๋วยเตี๋ยวต้มยำทะเลน้ำ', 'ก๋วยเตี๋ยวต้มยำทะเลแห้ง', 'ก๋วยเตี๋ยวสุโขทัยแห้ง', 'ก๋วยเตี๋ยวสุโขทัยน้ำ']

for item in items_to_clean:
    df['รายละเอียดรายการ'] = df['รายละเอียดรายการ'].str.replace(item, item.replace('ก๋วยเตี๋ยว', '').strip())

# Display the updated dataframe to verify the changes
display(df[df['รายละเอียดรายการ'].isin(['ต้มยำทะเลน้ำ', 'ต้มยำทะเลแห้ง', 'สุโขทัยแห้ง', 'สุโขทัยน้ำ'])].head())

In [ ]:
def identify_special_orders(row):
    item = row['รายละเอียดรายการ']
    price = row['ราคา']
    item_type = row['ประเภท']

    if item_type == 'ก๋วยเตี๋ยว':
        if 'สุโขทัย' in item and price > 55:
            return 'พิเศษ'
        elif 'ต้มยำทะเล' in item and price > 70:
            return 'พิเศษ'
        elif 'ก๋วยเตี๋ยว' in item and price > 45:
            return 'พิเศษ'
    elif item_type == 'ตามสั่ง':
        if price > 68.25: # Using the average price calculated earlier
            return 'พิเศษ'
    return 'ปกติ'

df['ประเภทพิเศษ'] = df.apply(identify_special_orders, axis=1)

special_orders = df[df['ประเภทพิเศษ'] == 'พิเศษ']

print("รายการที่น่าจะมีการสั่งพิเศษ:")
display(special_orders.head())

# Count special orders by item
special_item_counts = special_orders.groupby('รายละเอียดรายการ')['จำนวน'].sum()

# Count total orders by item
total_item_counts = df.groupby('รายละเอียดรายการ')['จำนวน'].sum()

# Calculate the percentage of special orders
special_order_percentage = (special_item_counts / total_item_counts * 100).fillna(0).sort_values(ascending=False)

print("\nเปอร์เซ็นต์ของรายการที่น่าจะมีการสั่งพิเศษ (เรียงตามเปอร์เซ็นต์):")
display(special_order_percentage.head(10))

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# Plot a bar chart for the top 10 items with the highest percentage of special orders
plt.figure(figsize=(12, 8))
special_order_percentage.head(10).plot(kind='bar')
plt.title('Top 10 Items by Percentage of Special Orders')
plt.xlabel('Item Name')
plt.ylabel('Percentage of Special Orders (%)')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

#เมนูไหนบ้างที่คนมักจะสั่งน้ำหรือเครื่องเคียงเพิ่ม

In [ ]:
# Group by เลขที่ใบเสร็จ and get a list of item details and types in each order
order_items_details_and_types = df.groupby('เลขที่ใบเสร็จ').apply(lambda x: list(zip(x['รายละเอียดรายการ'], x['ประเภท'])))

# Filter orders that contain both 'ก๋วยเตี๋ยว' or 'ตามสั่ง' and 'เครื่องเคียง' or 'เครื่องดื่ม'
relevant_orders_details_and_types = order_items_details_and_types[order_items_details_and_types.apply(lambda x: any(item_type in ['ก๋วยเตี๋ยว', 'ตามสั่ง'] for item_detail, item_type in x) and any(item_type in ['เครื่องเคียง', 'เครื่องดื่ม'] for item_detail, item_type in x))]

# For these relevant orders, find the specific 'ก๋วยเตี๋ยว' or 'ตามสั่ง' item details
def get_main_item_details_with_additions(order_items_list):
    main_item_details = [item_detail for item_detail, item_type in order_items_list if item_type in ['ก๋วยเตี๋ยว', 'ตามสั่ง']]
    return main_item_details

main_item_details_in_relevant_orders = relevant_orders_details_and_types.apply(get_main_item_details_with_additions)

# Count the occurrences of each specific main item detail in these orders
main_item_addition_details_counts = pd.Series([item for sublist in main_item_details_in_relevant_orders for item in sublist]).value_counts()

print("จำนวนครั้งที่เมนูก๋วยเตี๋ยวหรือตามสั่งถูกสั่งพร้อมเครื่องเคียงหรือเครื่องดื่ม (เรียงจากมากไปน้อย):")
display(main_item_addition_details_counts)

# To find which specific เครื่องเคียง or เครื่องดื่ม are ordered with which main items,
# we would need a more complex Market Basket Analysis or pairwise analysis.
# The current approach counts how often a main item appears in orders with additions.

In [ ]:
# Calculate the percentage of each main item ordered with additions
# Use main_item_addition_details_counts from the previous cell and item_counts calculated earlier
main_item_addition_percentage = (main_item_addition_details_counts / item_counts * 100).fillna(0).sort_values(ascending=False)

print("เปอร์เซ็นต์ของเมนูก๋วยเตี๋ยวหรือตามสั่งที่ถูกสั่งพร้อมเครื่องเคียงหรือเครื่องดื่ม (เรียงจากมากไปน้อย):")
display(main_item_addition_percentage.head(30))

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# Plot a bar chart for the top 30 items with the highest percentage of additions
plt.figure(figsize=(12, 8))
main_item_addition_percentage.head(30).plot(kind='bar')
plt.title('Top 30 Main Items by Percentage Ordered with Additions')
plt.xlabel('Item Name')
plt.ylabel('Percentage Ordered with Additions (%)')
plt.xticks(rotation=90)
plt.tight_layout()
plt.show()

#ลูกค้ามากันกี่คนมีแนวโน้มจะสั่งเครื่องเคียงหรือน้ำเพิ่ม

In [ ]:
import matplotlib.pyplot as plt

# Get the list of เลขที่ใบเสร็จ from the relevant_orders_details_and_types Series
relevant_order_ids = relevant_orders_details_and_types.index

# Filter the original DataFrame to include only these relevant orders
relevant_orders_df = df[df['เลขที่ใบเสร็จ'].isin(relevant_order_ids)]

# Plot a histogram of the 'จำนวนลูกค้า' column for these relevant orders
plt.figure(figsize=(10, 6))
plt.hist(relevant_orders_df['จำนวนลูกค้า'], bins=range(1, relevant_orders_df['จำนวนลูกค้า'].max() + 1), align='left', rwidth=0.8)
plt.xticks(range(1, relevant_orders_df['จำนวนลูกค้า'].max() + 1))
plt.xlabel('Number of Customers')
plt.ylabel('Frequency of Orders with Additions')
plt.title('Distribution of Customer Group Size in Orders with Additions')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

In [ ]:
# Group the original DataFrame by 'จำนวนลูกค้า' and count the total number of orders for each group size
total_orders_by_group_size = df.groupby('จำนวนลูกค้า')['เลขที่ใบเสร็จ'].nunique()

# Get the list of เลขที่ใบเสร็จ from the relevant_orders_details_and_types Series (orders with additions)
relevant_order_ids = relevant_orders_details_and_types.index

# Filter the original DataFrame to include only these relevant orders
relevant_orders_df = df[df['เลขที่ใบเสร็จ'].isin(relevant_order_ids)]

# Group the relevant orders DataFrame by 'จำนวนลูกค้า' and count the number of orders with additions for each group size
orders_with_additions_by_group_size = relevant_orders_df.groupby('จำนวนลูกค้า')['เลขที่ใบเสร็จ'].nunique()

# Calculate the percentage of orders with additions for each group size
percentage_orders_with_additions = (orders_with_additions_by_group_size / total_orders_by_group_size * 100).fillna(0).sort_index()

print("เปอร์เซ็นต์ของออเดอร์ที่มีการสั่งเครื่องเคียงหรือเครื่องดื่มเพิ่ม แบ่งตามจำนวนลูกค้า:")
display(percentage_orders_with_additions)

# You can also visualize this data if needed, for example, with a bar chart.
# plt.figure(figsize=(10, 6))
# percentage_orders_with_additions.plot(kind='bar')
# plt.xlabel('Number of Customers')
# plt.ylabel('Percentage of Orders with Additions (%)')
# plt.title('Percentage of Orders with Additions by Customer Group Size')
# plt.xticks(rotation=0)
# plt.grid(axis='y', linestyle='--', alpha=0.7)
# plt.show()

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# Create a bar chart
plt.figure(figsize=(10, 6))
percentage_orders_with_additions.plot(kind='bar')
plt.xlabel('Number of Customers')
plt.ylabel('Percentage of Orders with Additions (%)')
plt.title('Percentage of Orders with Additions by Customer Group Size')
plt.xticks(rotation=0)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.show()

##Market Basket Analysis

In [ ]:
!pip install mlxtend
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules

# Prepare the data for Market Basket Analysis
# We need a one-hot encoded DataFrame where each row is an order (เลขที่ใบเสร็จ)
# and each column is an item (รายละเอียดรายการ), with a value of 1 if the item
# is in the order and 0 otherwise.

# Group items by order and put them in a list
transactions = df.groupby('เลขที่ใบเสร็จ')['รายละเอียดรายการ'].apply(list).reset_index()

# Create a one-hot encoded DataFrame
# First, create a list of unique items
all_items = sorted(list(df['รายละเอียดรายการ'].unique()))

# Create a DataFrame of zeros with orders as index and items as columns
one_hot_df = pd.DataFrame(0, index=transactions['เลขที่ใบเสร็จ'], columns=all_items)

# Populate the one-hot encoded DataFrame
for index, row in transactions.iterrows():
    order_id = row['เลขที่ใบเสร็จ']
    items_in_order = row['รายละเอียดรายการ']
    for item in items_in_order:
        if item in one_hot_df.columns: # Check if item is in columns (should be as we used unique items)
             one_hot_df.loc[order_id, item] = 1

# Apply the Apriori algorithm
# Adjust the min_support as needed based on your data and desired frequency
frequent_itemsets = apriori(one_hot_df, min_support=0.01, use_colnames=True)

# Generate association rules
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

# Display the frequent itemsets and association rules
print("Frequent Itemsets:")
display(frequent_itemsets)

print("\nAssociation Rules:")
display(rules)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import numpy as np
import matplotlib.patches as mpatches # Import mpatches for custom legend handles

# Sort rules by lift to highlight the most interesting ones
rules_sorted_by_lift = rules.sort_values(by='lift', ascending=False).reset_index(drop=True)

# Select top N rules for plotting (adjust N as needed)
top_n_rules = rules_sorted_by_lift.head(10) # Let's plot top 10 rules

# Increase figure width
plt.figure(figsize=(15, 8))

# Create a list of unique item pairs in the top N rules
item_pairs = [f'{list(row["antecedents"])} -> {list(row["consequents"])}' for index, row in top_n_rules.iterrows()]
unique_item_pairs = list(dict.fromkeys(item_pairs)) # Get unique pairs while preserving order

# Assign a unique color to each unique item pair
colors = plt.cm.get_cmap('tab10', len(unique_item_pairs)) # Using a colormap

# Plot each rule with color based on the item pair and size based on lift
scatter_handles = [] # To store handles for the legend
for index, row in top_n_rules.iterrows():
    item_pair_str = f'{list(row["antecedents"])} -> {list(row["consequents"])}'
    color_index = unique_item_pairs.index(item_pair_str)
    plt.scatter(row['support'], row['confidence'], s=row['lift']*100, color=colors(color_index))

    # Create a legend handle with only color for each unique item pair
    if item_pair_str not in [h.get_label() for h in scatter_handles]:
        patch = mpatches.Patch(color=colors(color_index), label=item_pair_str)
        scatter_handles.append(patch)


# Add labels and title
plt.xlabel('Support')
plt.ylabel('Confidence')
plt.title('Association Rules: Support vs. Confidence (Colored by Item Pair, Size by Lift)')
plt.grid(True)

# Create a legend for the item pairs using the custom handles
plt.legend(handles=scatter_handles, title='Item Pairs', bbox_to_anchor=(1.05, 1), loc='upper left', labelspacing=1.5)


plt.tight_layout(rect=[0, 0, 0.8, 1]) # Adjust layout to make room for the legend

plt.show()

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# Create a directed graph
G = nx.DiGraph()

# Add nodes (items) to the graph
# Correctly combine antecedents and consequents
all_itemsets = pd.concat([rules['antecedents'], rules['consequents']])
for itemset in all_itemsets:
    for item in itemset:
        G.add_node(item)


# Add edges (rules) to the graph
for index, row in rules.iterrows():
    antecedent = list(row['antecedents'])
    consequent = list(row['consequents'])
    # For simplicity in visualization, let's assume each rule connects the first item
    # of the antecedent to the first item of the consequent.
    # For more complex rules (multiple items in antecedent/consequent),
    # you might need a different representation.
    if len(antecedent) > 0 and len(consequent) > 0:
        # Choose a representative item for the edge if antecedent/consequent are itemsets
        # Here we take the first item, you might need a different logic
        from_item = list(row['antecedents'])[0]
        to_item = list(row['consequents'])[0]
        G.add_edge(from_item, to_item, lift=row['lift'], confidence=row['confidence'], support=row['support'])


# Draw the graph
plt.figure(figsize=(12, 10))
pos = nx.spring_layout(G, k=0.5, iterations=50)  # Use spring layout for better visualization

# Draw nodes
nx.draw_networkx_nodes(G, pos, node_size=3000, node_color='skyblue', alpha=0.9)

# Draw edges with varying thickness based on lift
edge_widths = [d['lift'] for (u, v, d) in G.edges(data=True)]
# Normalize edge widths for better visualization if needed
max_width = max(edge_widths) if edge_widths else 1
normalized_widths = [w / max_width * 5 for w in edge_widths] # Scale to a max width of 5


nx.draw_networkx_edges(G, pos, edgelist=G.edges(), width=normalized_widths, arrowstyle='->', arrowsize=20, edge_color='gray', alpha=0.6)

# Draw labels
nx.draw_networkx_labels(G, pos, font_size=10, font_family='Sarabun')

# Add title
plt.title('Association Rules Network Graph (Edge Width by Lift)')
plt.axis('off') # Hide axes
plt.show()

In [ ]:
rules.to_excel("Market_Basket_rules.xlsx", index=False)

print("บันทึกไฟล์ Market_Basket_rules.xlsx เรียบร้อยแล้ว")